<a href="https://colab.research.google.com/github/Bast-aleja/Marketing_1/blob/main/Exploraci%C3%B3n_Trabajo_peliculasreal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.PROBLEMA DE NEGOCIO 
1.1 Descripción del problema de negocio

Esta plataforma online tiene inconvenientes en cuanto a la fidelización de clientes debido a que las demás plataformas brindan mejores experiencias dándole gusto al usuario y esta plataforma aún no lo ha empleado una estrategia competitiva y por ende el tema de tener nuevos clientes es más complejo.

1.2 Decripción del problema analítico

1.	Analizar el comportamiento y los hábitos de sus consumidores en la plataforma online
2.	Predecir contenido afín (películas) a los gustos que muestra el consumo que ha tenido el usuario.

1.3 Diseño de la solución 

En esta parte se realiza un diagrama de flujo con el paso a paso de la solución propuesta para este problema del área de marketingç:

[ENLACE AL GRAFICO](https://drive.google.com/file/d/1M_FLHt6C1PInxmPHR9BeY414EUk8m2sc/view?usp=sharing)

**CODIGO**

Importación de paquetes

In [ ]:
#Importación de paquetes
import pandas as pd
import numpy as np
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import requests


Lectura de datos

In [ ]:
#Conexión con colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
conn = sql.connect('db_movies')
cur= conn.cursor()

In [ ]:
#Leer cantidad de bases que tiene el archivo
tables = pd.read_sql("SELECT name FROM sqlite_master where type = 'table'", conn)
tables

,name
0,ratings
1,movies


Lectura base de datos: movies

In [ ]:
movies = pd.read_sql("SELECT * FROM movies", conn)
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


Depuración BD movies

In [ ]:
#Duplicados
movies.duplicated().sum()

0

In [ ]:
#Analizar dimensión, tipos y nulos
print(movies.shape)
print('-------------------------------------------------')
print(movies.isnull().sum())
print('-------------------------------------------------')
print(movies.dtypes)
print('-------------------------------------------------')

(9742, 3)
-------------------------------------------------
movieId    0
title      0
genres     0
dtype: int64
-------------------------------------------------
movieId     int64
title      object
genres     object
dtype: object
-------------------------------------------------


In [ ]:
#separa columna generos 


from mlxtend.preprocessing import TransactionEncoder

conn= sql.connect('db_movies')

movies = pd.read_sql('SELECT * FROM movies', conn)
genres=movies['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genress = pd.DataFrame(genres, columns = te.columns_)
genress


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,False,True,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
9738,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
9739,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
9740,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Unión categoria genres a la base movies

In [ ]:
movies.drop(columns=['genres'], inplace =True)


In [ ]:
movies = pd.concat ([movies, genress], axis = 1)

In [ ]:
#relacionar los verdaderos con 1 y los falsos con 0
dic_genres = {
    False:0,
    True:1
}
movies[genress.columns] = movies[genress.columns].replace(dic_genres )
movies

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,No Game No Life: Zero (2017),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint (2017),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Se arregla los tipos de datos de las columnas
movies=movies.convert_dtypes()
movies.dtypes

movieId                Int64
title                 string
(no genres listed)     Int64
Action                 Int64
Adventure              Int64
Animation              Int64
Children               Int64
Comedy                 Int64
Crime                  Int64
Documentary            Int64
Drama                  Int64
Fantasy                Int64
Film-Noir              Int64
Horror                 Int64
IMAX                   Int64
Musical                Int64
Mystery                Int64
Romance                Int64
Sci-Fi                 Int64
Thriller               Int64
War                    Int64
Western                Int64
dtype: object

In [ ]:
#Cambiar el nombre de las columnas a minusculas
movies.columns = movies.columns.str.lower()
movies.columns

Index(['movieid', 'title', '(no genres listed)', 'action', 'adventure',
       'animation', 'children', 'comedy', 'crime', 'documentary', 'drama',
       'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery',
       'romance', 'sci-fi', 'thriller', 'war', 'western'],
      dtype='object')

In [ ]:
#Separar el año
movies['y'] = movies['title'].str.split(' ')
movies['year'] = movies['y'].apply(lambda x: x[-1])
movies = movies.drop(['y'], axis = 1)
movies['year'] = movies['year'].str.strip(')')
movies['year'] = movies['year'].str.strip('(')


In [ ]:
#Mirar si existen datos nulos
movies['year'].isnull().sum()


0

In [ ]:
#Mirar si existen años mal escritos
movies['year'].unique()

array(['1995', '1994', '1996', '1976', '1992', '1967', '1993', '1964',
       '1977', '1965', '1982', '1990', '1991', '1989', '1937', '1940',
       '1969', '1981', '1973', '1970', '1955', '1959', '1968', '1988',
       '1997', '1972', '1943', '1952', '1951', '1957', '1961', '1958',
       '1954', '1934', '1944', '1960', '1963', '1942', '1941', '1953',
       '1939', '1950', '1946', '1945', '1938', '1947', '1935', '1936',
       '1956', '1949', '1932', '1975', '1974', '1971', '1979', '1987',
       '1986', '1980', '1978', '1985', '1966', '1962', '1983', '1984',
       '1948', '1933', '1931', '1922', '1998', '1929', '1930', '1927',
       '1928', '1999', '2000', '1926', '1919', '1921', '1925', '1923',
       '2001', '2002', '2003', '1920', '1915', '1924', '2004', '1916',
       '1917', '', '2005', '2006', '1902', '5', '1903', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '1908', 'One', 'Road', 'Watson', 'Animals',
       'P

In [ ]:
#Cuantos datos representan 
movies.query("year=='' or year=='5' or year== 'One' or year=='Road'or year=='Watson' or year=='Animals' or year=='Paterson'or year=='Moonlight' or year=='Moonlight' or year=='OA' or year=='Cosmos' or year=='Baby' or year=='20006-2007' or year=='2' or year=='Mirror'")

,movieid,title,(no genres listed),action,adventure,animation,children,comedy,crime,documentary,...,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western,year
5609,27008,From Dusk Till Dawn 2: Texas Blood Money (1999),0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,
6059,40697,Babylon 5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,5
6706,58404,Justice League: The New Frontier (2008),0,1,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,
6718,58842,Assembly (Ji jie hao) (2007),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,
7878,94494,96 Minutes (2011),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,
7896,95004,Superman/Doomsday (2007),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
7910,95193,Pocahontas II: Journey to a New World (1998),0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,
7978,96608,Runaway Brain (1995),0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,
8148,102084,Justice League: Doom (2012),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
8228,104017,3 dev adam (Three Giant Men) (1973),0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,


In [ ]:
#Se reemplazan los 23 datos en years escritos incorrectamente
lista= ('Cosmos', 'OA', 'One', 'Paterson', 'Moonlight', 'Watson', '2', 'Animals', 'Baby', 'Mirror','Road','','5')


Se encuentra que existen muy pocos datos (23 de 9742) así que se puede optar por borrarlos o reemplazar sus valores. Se tomara la media de los años y serán reemplazados por ese año ya que los nombres escritos no dan indicios de una fecha excepto por 2006-2007 que se tomara el 2007

In [ ]:
#Busco el dato que más se repite para que reemplace los valores

movies['year'].describe()

count    9742.000000
mean     1994.009957
std        18.926438
min      1902.000000
25%      1987.000000
50%      1998.000000
75%      2008.000000
max      2018.000000
Name: year, dtype: float64

In [ ]:
#Reemplazar los datos que no son fechas

movies['year'] = movies['year'].replace(('2006–2007','2006'))

for i in lista:
  movies['year'] = movies['year'].replace((i, 1994))



In [ ]:
#poner el año en el tipo de dato adecuado
movies['year'] = movies['year'].astype('int64')

In [ ]:
#Homologación de categorias 
listaf= ['title',
        '(no genres listed)', 'action', 'adventure',
       'animation', 'children', 'comedy', 'crime', 'documentary', 'drama',
       'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery',
       'romance', 'sci-fi', 'thriller', 'war', 'western', 'year']

In [ ]:
for i in listaf:
  print(movies[i].value_counts())
  print('-----------------------------------------------')

Lectura base de datos: raitings

In [ ]:
#Lectura DB ratingS
ratings=pd.read_sql("""select *,datetime(timestamp , 'unixepoch') as date from ratings""", conn)
ratings

,userId,movieId,rating,timestamp,date
0,1,1,4.0,964982703,2000-07-30 18:45:03
1,1,3,4.0,964981247,2000-07-30 18:20:47
2,1,6,4.0,964982224,2000-07-30 18:37:04
3,1,47,5.0,964983815,2000-07-30 19:03:35
4,1,50,5.0,964982931,2000-07-30 18:48:51
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,2017-05-03 21:53:22
100832,610,168248,5.0,1493850091,2017-05-03 22:21:31
100833,610,168250,5.0,1494273047,2017-05-08 19:50:47
100834,610,168252,5.0,1493846352,2017-05-03 21:19:12


**Depuración DB ratings**

In [ ]:
#información general
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
 4   date       100836 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 3.8+ MB


In [ ]:
#Duplicados
ratings.duplicated().sum()

0

In [ ]:
#Analizar dimensión, tipos y nulos
print(ratings.shape)
print('-------------------------------------------------')
print(ratings.isnull().sum())
print('-------------------------------------------------')
print(ratings.dtypes)
print('-------------------------------------------------')

(100836, 5)
-------------------------------------------------
userId       0
movieId      0
rating       0
timestamp    0
date         0
dtype: int64
-------------------------------------------------
userId         int64
movieId        int64
rating       float64
timestamp      int64
date          object
dtype: object
-------------------------------------------------


In [ ]:
#Se arregla los tipos de datos de las columnas
ratings=ratings.convert_dtypes()
ratings.dtypes

userId         Int64
movieId        Int64
rating       Float64
timestamp      Int64
date          string
dtype: object

In [ ]:
#Se le quita la hora a la fecha
ratings['date']=ratings['date'].apply(lambda x:str(x)[0:10])

In [ ]:
#Se le quita la columna timestap ya que al convertirla a date ya no es necesaria
ratings=ratings.drop(['timestamp'], axis=1)

In [ ]:
#Se arregla el tipo de dato de date a fecha

ratings['date']= pd.to_datetime(ratings['date'], errors= 'coerce', format='%Y-%m-%d')
ratings.dtypes

userId                Int64
movieId               Int64
rating              Float64
timestamp             Int64
date         datetime64[ns]
dtype: object

In [ ]:
#Cambiar el nombre de las columnas a minusculas
ratings.columns = ratings.columns.str.lower()
ratings.columns

Index(['userid', 'movieid', 'rating', 'timestamp', 'date'], dtype='object')

In [ ]:
#Homologación de categorias 
#timestap tiene la misma fecha para todos (no sirve?-eliminar?)
listaf= ['rating', 'date']

In [ ]:
for i in listaf:
  print(ratings[i].value_counts())
  print('-----------------------------------------------')

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: Int64
-----------------------------------------------
2017-06-26    1014
2007-05-14     878
2017-05-03     866
2000-08-08     709
2015-06-28     606
              ... 
2010-01-09       1
2005-03-27       1
2005-08-14       1
2003-10-29       1
2018-03-08       1
Name: date, Length: 4110, dtype: int64
-----------------------------------------------


In [ ]:
#Se guardan las bases de datos finales
movies = movies.copy(deep = True)
ratings =ratings.copy(deep= True)



## Análisis exploratorio






In [ ]:
 #Paquetes
import plotly.express as  px
import numpy as np
import datetime

**Análisis exploratorio movies**

In [ ]:
movies.columns

Index(['movieid', 'title', '(no genres listed)', 'action', 'adventure',
       'animation', 'children', 'comedy', 'crime', 'documentary', 'drama',
       'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery',
       'romance', 'sci-fi', 'thriller', 'war', 'western', 'year'],
      dtype='object')

1. Número de peliculas que se tuvo por año

In [ ]:
G1= movies.groupby(['year'], sort=True)[['title']].count().reset_index()
G1_1= G1.rename(columns={'title':'Frecuencia'})

In [ ]:
#Gráfico
fig  =  px . bar ( G1_1,  x = 'year' ,  y = 'Frecuencia', title='Número de peliculas por año') 
fig.update_layout(
    title_x = 0.5)
fig . show ()

2. Número de generos que tuvieron las peliculas

In [ ]:
G2=movies.drop(['year', 'movieid', '(no genres listed)','title'], axis=1)
G2_1=G2.sum()
G2_2 = pd.DataFrame(G2_1, columns=['frecuencia']).reset_index()
G2_3 = G2_2.rename(columns={'index':'Generos'})
G2_3.sort_values(by="frecuencia", ascending=False)

,Generos,frecuencia
7,drama,4361
4,comedy,3756
16,thriller,1894
0,action,1828
14,romance,1596
1,adventure,1263
5,crime,1199
15,sci-fi,980
10,horror,978
8,fantasy,779


In [ ]:
#Gráfico
fig  =  px . bar ( G2_3,  x = 'Generos' ,  y = 'frecuencia', title='Géneros por pelicula') 
fig.update_layout(
    title_x = 0.5)
fig . show ()

3. Cuantos generos tiene cada pelicula (primeras 10 con más generos)

In [ ]:
G3=movies.drop(['year', 'movieid', '(no genres listed)'], axis=1)
G3['suma_filas']=G3.sum(axis=1)
G3_1=G3.drop([ 'action', 'adventure',
       'animation', 'children', 'comedy', 'crime', 'documentary', 'drama',
       'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery',
       'romance', 'sci-fi', 'thriller', 'war', 'western'], axis=1)
G3_1.sort_values(by="suma_filas", ascending=False).head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,title,suma_filas
7441,Rubber (2010),10.0
5556,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,8.0
3460,Osmosis Jones (2001),7.0
6626,Enchanted (2007),7.0
4631,Interstate 60 (2002),7.0
1390,Mulan (1998),7.0
2250,Who Framed Roger Rabbit? (1988),7.0
5819,Robots (2005),7.0
6145,Pulse (2006),7.0
6462,Aqua Teen Hunger Force Colon Movie Film for Th...,7.0


**Análisis exploratorio ratings**

In [ ]:
ratings.columns

Index(['userid', 'movieid', 'rating', 'date'], dtype='object')

1. Conteo de las calificaciones obtenidas

In [ ]:
#Descripción base de ratings
co=pd.read_sql(""" select 
                          "rating" as ratings, 
                          count(*) as conteo 
                          from ratings
                          group by "rating"
                          """, conn)

In [ ]:
#Gráfica
data  = go.Bar( x=co.ratings,y=co.conteo, text=co.conteo, textposition="outside")
Layout=go.Layout(title="Conteo de calificaciones",xaxis={'title':'Calificación'},yaxis={'title':'Total'})
go.Figure(data,Layout)

2. Peliculas vistas por usuario

In [ ]:
R2=pd.read_sql(''' select userId,
                         count(*) as cont
                         from ratings
                         group by "userId"
                         order by cont desc
                         ''',conn )


3. Número de peliculas vistas por año

In [ ]:
ratings1=ratings
ratings1['Años'] = ratings['date'].dt.year
ratings1['Años'].value_counts()
R3=ratings1.drop(['userid', 'rating', 'date', 'movieid'], axis=1)
R3_1=R3.value_counts()
R3_2 = pd.DataFrame(R3_1).reset_index()
R3_3 = R3_2.rename(columns={0 :'Frecuencia'})
R3_3

In [ ]:
#Gráfico
fig  =  px . bar ( R3_3,  x = 'Años' ,  y = 'Frecuencia', title='Conteo de peliculas vistas por año') 
fig.update_layout(
    title_x = 0.5)
fig . show ()

4. Número de peliculas vistas por mes

In [ ]:
ratings1['Mes'] = ratings['date'].dt.month
R4=ratings1.drop(['userid', 'rating', 'date', 'movieid', 'Años'], axis=1)
R4_1=R4.value_counts()
R4_2 = pd.DataFrame(R4_1).reset_index()
R4_3 = R4_2.rename(columns={0 :'Frecuencia'})
R4_3

In [ ]:
#Gráfico
fig  =  px . bar ( R4_3,  x = 'Mes' ,  y = 'Frecuencia', title='Conteo de peliculas vistas por mes') 
fig.update_layout(
    title_x = 0.5)
fig . show ()

5. Los usuarios con más de 1000 peliculas vistas

In [ ]:
#usuarios con más de 1000 peliculas vistas
R5_1=pd.read_sql(""" select 
                          "userId" as usuario, 
                          count(*) as contar 
                          from ratings                        
                          group by "userId"
                          order by contar desc
                          """, conn)
R5_1

In [ ]:
R5_2=R5_1[(R5_1['contar']>1000)]
R5_2

,usuario,contar
0,414,2698
1,599,2478
2,474,2108
3,448,1864
4,274,1346
5,610,1302
6,68,1260
7,380,1218
8,606,1115
9,288,1055


6. Las peliculas con mejor calificadas y con más de 100 calificaciones

In [ ]:
#usuarios con más de 1000 peliculas vistas
R6_1=pd.read_sql(""" select 
                          movieid, rating,
                          count(*) as cnt
                          from ratings
                          where rating >4.0
                          group by movieid
                          order by cnt desc
                          """, conn)
R6_1

In [ ]:
R6_2=R6_1[(R6_1['cnt']>100)]
R6_2

,movieId,rating,cnt
0,318,4.5,202
1,296,5.0,167
2,356,5.0,155
3,2571,5.0,150
4,2959,5.0,130
5,260,5.0,128
6,527,5.0,128
7,593,4.5,128
8,858,5.0,114
9,50,4.5,113


**Análisis de los resultados**



1. Las películas que contienen Drama o comedia son aquellas que más se realizan en el cine

2. Se evidencia que el número de generos que tenga una pelicula no significa que intervenga o haga que más usuarios la miren, esto se nota debido a que ninguna de las peliculas que tuvo más generos estuvo entre las mejores calificadas o más vistas

3. se nota que existe una buena acogida por parte de los usuarios a las películas que se tiene en la plataforma pues son pocos en comparación los que le dan una calificación menor a 3

4. La gráfica donde podemos evidenciar los meses en los cuales se miran más películas se podrían interpretar debido al publico en el que se están tomando los datos. Por lo que, para la epoca de finales de mayo y noviembre empiezan sus vacaciones que es donde más tienen tiempo libre, pero, en las vacaciones de junio y diciembre se podrían dedicar a viajar o las celebraciones tipicas de diciembre más que a mirar peliculas

5. Con la gráfica de los años en donde más peliculas se miraron podemos observar que existe un salto significativo de 1999 al 2000 y esto se podría inferir que fue debido a que en esta epoca se introduce fuertemente la conexión inalámbrica, La nube y el internet de banda ancha. También se puede deber a que para esa fecha aumento el nivel de peliculas existentes según la gráfica G1_1

6. Con la gráfica de los años tambien se puede interpretar el elevado crecimiento de las peliculas vistas desde el 2015 debido a que las generaciones que más empezarón a depender de las pantallas y a utilizar el internet en gran medida fuerón aquellos que nacieron entre 1999 en adelante por lo que para el 2015 ya tenían la edad suficiente para manejar el tiempo con libertad y aumentar está practica de mirar peliculas en su tiempo libre

7. Con la base de datos R5_2 se podría implementar un beneficio a aquellos usuarios que han sido destacados con un alto porcentaje de peliculas vistas cada mes aumentando así la fidelización del cliente y proponiendo nuevas estrategias de marketing que cautiven la atención de más clientes

8. Con la base de datos R6_2 se obtienen no solo las peliculas más calificadas sino también aquellas que mejor fueron calificadas, por lo que se podría implementar las recomendaciones de estas peliculas a un publico general al ser el top de la plataforma, ya que tienen buenos antecedentes para que gusten a los usuraios y más quieran verlas
